In [ ]:
!pip install openpyxl

In [ ]:
!pip install autogluon.tabular

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 47.4 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pandas<2.2.2dev0,>=2.0, but you have pandas 2.2.3 which is incompatible.
google-colab 1.0.0 requires pandas==2.1.4, but you have pandas 2.2.3 which is incompatible.
lightautoml 0.3.8.1 requires pandas<2.0.0, but you have pandas 2.2.3 which is incompatible.


In [ ]:
!pip install -U lightautoml

  Using cached pandas-1.5.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
Using cached pandas-1.5.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.1 MB)
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.3
    Uninstalling pandas-2.2.3:
      Successfully uninstalled pandas-2.2.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-common 1.1.1 requires pandas<2.3.0,>=2.0.0, but you have pandas 1.5.3 which is incompatible.
autogluon-core 1.1.1 requires pandas<2.3.0,>=2.0.0, but you have pandas 1.5.3 which is incompatible.
autogluon-features 1.1.1 requires pandas<2.3.0,>=2.0.0, but you have pandas 1.5.3 which is incompatible.
autogluon-tabular 1.1.1 requires pandas<2.3.0,>=2.0.0, but you have pandas 1.5.3 which is incompatible.
cudf-cu12 24.4.1 requires pandas<2.2.2dev0,>=2.0, but you hav

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
from lightautoml.automl.presets.tabular_presets import TabularAutoML
from lightautoml.tasks import Task
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from autogluon.tabular import TabularPredictor

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_classif
from imblearn.over_sampling import SMOTE  # Для балансировки классов
from sklearn.preprocessing import StandardScaler

In [ ]:
train_df = pd.read_excel('/content/drive/MyDrive/Datasets/train.xlsx', index_col='Unnamed: 0')
test_df = pd.read_excel('/content/drive/MyDrive/Datasets/test.xlsx', index_col='Unnamed: 0')

In [ ]:
# Предобработка данных
def date_to_features(df):
    df['Дата бронирования'] = pd.to_datetime(df['Дата бронирования'])
    df['Заезд'] = pd.to_datetime(df['Заезд'])
    df['Выезд'] = pd.to_datetime(df['Выезд'])

    df['день_бронирования'] = df['Дата бронирования'].dt.day
    df['месяц_бронирования'] = df['Дата бронирования'].dt.month
    df['год_бронирования'] = df['Дата бронирования'].dt.year
    df['день_заезда'] = df['Заезд'].dt.day
    df['месяц_заезда'] = df['Заезд'].dt.month
    df['год_заезда'] = df['Заезд'].dt.year
    df['день_выезда'] = df['Выезд'].dt.day
    df['месяц_выезда'] = df['Выезд'].dt.month
    df['год_выезда'] = df['Выезд'].dt.year
    df['разница_заезда'] = (df['Заезд'] - df['Дата бронирования']).dt.days

    # Новые признаки: сезон года и день недели
    df['сезон_бронирования'] = df['месяц_бронирования'].apply(lambda x: (x % 12 + 3) // 3)
    df['день_недели_бронирования'] = df['Дата бронирования'].dt.weekday
    df['день_недели_заезда'] = df['Заезд'].dt.weekday
    df['длительность_пребывания'] = (df['Выезд'] - df['Заезд']).dt.days
    return df

train_df = date_to_features(train_df)
test_df = date_to_features(test_df)

# Выбираем признаки
features = ['Номеров', 'Стоимость', 'Внесена предоплата', 'Способ оплаты',
            'день_бронирования', 'месяц_бронирования', 'год_бронирования',
            'день_заезда', 'месяц_заезда', 'год_заезда',
            'день_выезда', 'месяц_выезда', 'год_выезда',
            'Гостей', 'Гостиница', 'разница_заезда', 'сезон_бронирования',
            'день_недели_бронирования', 'день_недели_заезда', 'длительность_пребывания']
target = 'Дата отмены'

train_df['target'] = train_df[target].notnull().astype(int)

X = train_df[features]
y = train_df['target']

# Определяем категориальные и числовые признаки
categorical_features = ['Способ оплаты', 'Гостиница']
numeric_features = [col for col in features if col not in categorical_features]

# Предобработчик для категориальных признаков
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),  # Стандартизация числовых признаков
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)  # Кодирование категориальных признаков
    ]
)

# Преобразуем данные перед применением SMOTE
X_transformed = preprocessor.fit_transform(X)

# Балансировка классов с использованием SMOTE
smote = SMOTE(random_state=42)
X_balanced, y_balanced = smote.fit_resample(X_transformed, y)

In [ ]:
predictor = TabularPredictor(label='Дата отмены', eval_metric='roc_auc').fit(train_df, presets='good_quality')

No path specified. Models will be saved in: "AutogluonModels/ag-20240922_084501"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          2
Memory Avail:       10.46 GB / 12.67 GB (82.5%)
Disk Space Avail:   68.23 GB / 107.72 GB (63.3%)
Presets specified: ['good_quality']


NotImplementedError: label dtype datetime64[ns] not supported!

In [ ]:
# Оцениваем модель на тестовой выборке
performance = predictor.evaluate(test_data)

In [ ]:
# Выводим метрики производительности
print(performance)

{'roc_auc': 0.8628007140055984, 'accuracy': 0.8850574712643678, 'balanced_accuracy': 0.7529006477437152, 'mcc': 0.6461983329914138, 'f1': 0.6666666666666666, 'precision': 0.9493670886075949, 'recall': 0.5136986301369864}


In [ ]:
# Делаем предсказания на тестовой выборке
predictions = predictor.predict(df_test)

In [ ]:
predictions

,Дата отмены
0,0
1,0
2,0
3,0
4,0
...,...
11213,0
11214,0
11215,0
11216,0


In [ ]:
# Преобразуем predictions_binary в Pandas Series для использования value_counts()
predictions_binary_series = pd.Series(predictions)

# Подсчитываем количество каждого уникального значения
score_counts = predictions_binary_series.value_counts()

# Выводим уникальные значения и их количество
print(score_counts)

Дата отмены
0    10038
1     1180
Name: count, dtype: int64


In [ ]:
# Сохраняем бинарные предсказания в файл
submission_df = pd.DataFrame(predictions)
submission_df.to_csv('/content/drive/MyDrive/Karmanova/predictions_amazon.csv', index=False, header=False)